In [0]:
import pandas as pd
import numpy as np
import cv2
import os

In [2]:
!pip install kaggle
!mkdir -p ~/.kaggle

In [0]:
!cp kaggle.json ~/.kaggle/

In [5]:
!kaggle datasets download -d moltean/fruits

 98% 678M/690M [00:08<00:00, 88.3MB/s]
100% 690M/690M [00:08<00:00, 87.7MB/s]


In [0]:
!unzip fruits.zip

In [0]:
directory1 = "fruits-360/Training/"
classes=["Apple Golden 1","Avocado","Banana","Cherry 1","Cocos","Kiwi","Lemon","Mango","Orange"]
train=[]
img_size=100
for fruit in classes:
    path = os.path.join(directory,fruit)
    class_num = classes.index(fruit)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(img_size,img_size))
        train.append([img_array,class_num])

In [0]:
directory2 = "fruits-360/Test/"
classes=["Apple Golden 1","Avocado","Banana","Cherry 1","Cocos","Kiwi","Lemon","Mango","Orange"]
test=[]
img_size=100
for fruit in classes:
    path = os.path.join(directory,fruit)
    class_num = classes.index(fruit)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img),cv2.IMREAD_GRAYSCALE)
        img_array = cv2.resize(img_array,(img_size,img_size))
        test.append([img_array,class_num])

In [0]:
import random

#training data
random.shuffle(train)

X_train=[]
Y_train=[]
for features,label in train:
    X_train.append(features)
    Y_train.append(label)
X_train = np.array(X_train)

#test data
random.shuffle(test)

X_test=[]
Y_test=[]
for features,label in test:
    X_test.append(features)
    Y_test.append(label)
X_test=np.array(X_test) 

In [13]:
X_train=X_train.reshape(-1,img_size,img_size,1)
X_train=X_train/255
X_test=X_test.reshape(-1,img_size,img_size,1)
X_test=X_test/255
print("shape of X_train= ",X_train.shape)
print("shape of X_test=  ",X_test.shape)

shape of X_train=  (4318, 100, 100, 1)
shape of X_test=   (4318, 100, 100, 1)


In [14]:
from keras.utils import to_categorical
Y_train=to_categorical(Y_train,num_classes=9)
Y_test=to_categorical(Y_test,num_classes=9)

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPool2D

In [16]:
model=Sequential()
model.add(Conv2D(filters=8,kernel_size=(5,5),padding="Same",activation="relu",input_shape=(100,100,1)))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=16,kernel_size=(4,4),padding="Same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=32,kernel_size=(4,4),padding="Same",activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(9,activation="softmax"))

W0619 14:03:43.275723 140405043693440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0619 14:03:43.322499 140405043693440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0619 14:03:43.330554 140405043693440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0619 14:03:43.383639 140405043693440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0619 14:03:43.387383 140405043693440 deprecation_wrapp

In [17]:
model.compile(optimizer = 'adam',loss = "categorical_crossentropy",metrics = ["accuracy"])

W0619 14:07:53.067353 140405043693440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0619 14:07:53.108280 140405043693440 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [18]:
model.fit(X_train,Y_train,batch_size = 36,epochs = 40,validation_data = (X_test,Y_test))

W0619 14:15:34.152849 140405043693440 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 4318 samples, validate on 4318 samples
Epoch 1/40
4318/4318 [==============================] - 51s 12ms/step - loss: 0.6819 - acc: 0.7529 - val_loss: 0.1166 - val_acc: 0.9706
Epoch 2/40
4318/4318 [==============================] - 51s 12ms/step - loss: 0.0922 - acc: 0.9667 - val_loss: 0.0138 - val_acc: 1.0000
Epoch 3/40
4318/4318 [==============================] - 51s 12ms/step - loss: 0.0266 - acc: 0.9919 - val_loss: 0.0068 - val_acc: 0.9995
Epoch 4/40
 144/4318 [>.............................] - ETA: 39s - loss: 0.0196 - acc: 1.0000

KeyboardInterrupt: ignored

In [20]:
model.predict_classes(X_test)

array([7, 6, 8, ..., 3, 6, 8])

In [21]:
type(X_test)

numpy.ndarray